In [53]:
w={1:{2:10, 3:25, 4:20, 5:17}}

from operator import itemgetter
#sorted(list1, key=itemgetter(1),reverse=True)
sorted(w[1].items(), key = lambda s: s[1],reverse=True)

[(3, 25), (4, 20), (5, 17), (2, 10)]

In [54]:
w={1:{2:10, 3:25, 4:20, 5:17}}
train_set = {1:['xx','bb','zz'],\
               2:['22','32','zz'],\
               3:['xx','2123'],\
               4:['88','zz'],\
               5:['88','22']}

print(type(train_set[1]))
def Recommend(user, train_set, W, k = 3):
    already_view_items = train_set[user] 
    rank = dict()
    for user_j, weight_user_ij in sorted(W[user].items(), key = lambda w_user_j: w_user_j[1],reverse=True)[:k]:
        for items in train_set[user_j]:
            if items in already_view_items:
                continue;
            else:
                rank.setdefault(items, 0)
                rank[items] += weight_user_ij
    return sorted(rank.items(), key = lambda w:w[1], reverse = True)
    
Recommend(1,train_set, w )    

<class 'list'>


[('88', 37), ('2123', 25), ('22', 17)]

In [52]:
def save_dict2json(file_name, dic): 
    import json
    with open(file_name+'.json','a') as outfile:
        json.dump(dic,outfile,ensure_ascii=False)
        outfile.write('\n')

def read_json2dict(file_path):
    import json
    data = []
    with  open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    f.close()
    if len(data) ==  1:
        return data[0]
    else:
        return data

In [55]:
save_dict2json('dict', train_set)

In [56]:
dic=read_json2dict('dict.json')

In [61]:
dic

{'1': ['xx', 'bb', 'zz'],
 '2': ['22', '32', 'zz'],
 '3': ['xx', '2123'],
 '4': ['88', 'zz'],
 '5': ['88', '22']}

和UserCF算法类似，用ItemCF算法计算物品相似度时也可以首先建立用户—物品倒排表（即
对每个用户建立一个包含他喜欢的物品的列表），然后对于每个用户，将他物品列表中的物品两
两在共现矩阵C中加1

In [84]:
train_set = {1:['xx','bb','zz'],\
               2:['22','32','zz','2123'],\
               3:['6666','2123'],\
               4:['88','zz'],\
               5:['88','22']}
#同一个用户，都访问过哪些item，
import math
def ItemSimilarity(train_set):
    item_sim_matrix = dict()
    N = dict()
    for user, items in train_set.items():
        for item_i in items:
            N.setdefault(item_i, 0)
            N[item_i] += 1
            for item_j in items:
                if(item_i == item_j):
                    continue
                item_sim_matrix.setdefault(item_i, {})
                item_sim_matrix[item_i].setdefault(item_j, 0)
                item_sim_matrix[item_i][item_j] += 1
                
    for item, related_items in item_sim_matrix.items():
        for item_j, cuv in related_items.items():
            item_sim_matrix[item][item_j] = cuv/math.sqrt((N[item] * (N[item_j])))                                                                  
    return item_sim_matrix  
            
W = ItemSimilarity(train_set)
print(W)
#还可以做归一化
def Recommendation(train, user_id, W, K):
    rank = dict()
    ru = train[user_id]
    for i,pi in ru.items():
        for j, wj in sorted(W[i].items(),\
            key=itemgetter(1), reverse=True)[0:K]:
            if j in ru:
                continue
            rank[j].weight += pi * wj
            rank[j].reason[i] = pi * wj
    return rank

def Recommendation2(train, user_id, W, K):
    rank = dict()
    ru = train[user_id]
    for i in ru:
        for j, wj in sorted(W[i].items(),\
            key=itemgetter(1), reverse=True)[0:K]:
            if j in ru:
                continue
            rank.setdefault(j, 0.0)
            rank[j] += 1.0 * wj

    return rank 

Recommendation2(train_set, 2, W, K=4)

{'xx': {'bb': 1.0, 'zz': 0.5773502691896258}, 'bb': {'xx': 1.0, 'zz': 0.5773502691896258}, 'zz': {'xx': 0.5773502691896258, 'bb': 0.5773502691896258, '22': 0.4082482904638631, '32': 0.5773502691896258, '2123': 0.4082482904638631, '88': 0.4082482904638631}, '22': {'32': 0.7071067811865475, 'zz': 0.4082482904638631, '2123': 0.5, '88': 0.5}, '32': {'22': 0.7071067811865475, 'zz': 0.5773502691896258, '2123': 0.7071067811865475}, '2123': {'22': 0.5, '32': 0.7071067811865475, 'zz': 0.4082482904638631, '6666': 0.7071067811865475}, '6666': {'2123': 0.7071067811865475}, '88': {'zz': 0.4082482904638631, '22': 0.5}}


{'88': 0.5,
 'xx': 0.5773502691896258,
 'bb': 0.5773502691896258,
 '6666': 0.7071067811865475}